<a href="https://colab.research.google.com/github/MDIMAM132202006/MTech-Project-Code/blob/Prediction-of-Track-Classification/classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier , RandomForestRegressor
import pandas as pd
get_ipython().run_line_magic('matplotlib', 'inline')
import seaborn as sns;

from sklearn.model_selection import train_test_split , GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay ,f1_score ,r2_score

from sklearn.svm import SVR

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load your dataset
df = pd.read_excel('balling and roughness (2).xlsx', sheet_name = 'Sheet3')
df

,Power,Velocity,layer thickness,beam D,depth of meltpool,width of melt pool,length of melt pool,depth/thickness,length/width,width/depth,...,Weber number,"surface tension, N/m",spatter,VED,p,pi,spatter.1,m,effective power,ref
0,200,850.0,30.0,90.0,110.000000,140.000000,757.902572,3.666667,5.413590,1.272727,...,0.762707,-237.920389,Spatter,37.004658,0.107712,243.608628,blank,Conduction Mode,102.319994,https://www.sciencedirect.com/science/article/...
1,200,1000.0,30.0,90.0,92.000000,130.000000,679.508923,3.066667,5.226992,1.413043,...,0.946459,-201.276391,Spatter,31.453959,0.091556,222.985317,blank,Conduction Mode,94.121044,NaN
2,200,1150.0,30.0,90.0,90.000000,105.000000,607.355145,3.000000,5.784335,1.166667,...,1.118781,-172.905271,Spatter,27.351269,0.079614,212.679234,blank,Conduction Mode,86.934645,NaN
3,200,1300.0,30.0,90.0,68.000000,105.000000,540.985910,2.266667,5.152247,1.544118,...,1.273442,-150.448638,Spatter,24.195353,0.070427,201.928468,blank,Conduction Mode,80.649475,NaN
4,200,1450.0,30.0,90.0,72.000000,105.000000,479.942895,2.400000,4.570885,1.458333,...,1.405504,-132.335280,Spatter,21.692386,0.063142,197.160224,blank,Conduction Mode,75.139529,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,500,40.0,250.0,300.0,1005.779644,1474.095190,3570.517737,4.023119,2.422176,1.465624,...,0.007957,-620.499381,no spatter,176.928521,0.686667,1123.165659,blank,Transition Mode,349.341321,NaN
297,500,50.0,250.0,300.0,867.461958,1348.371226,3296.355442,3.469848,2.444694,1.554387,...,0.011478,-552.006536,no spatter,141.542817,0.549333,947.743444,blank,Transition Mode,347.689251,NaN
298,500,60.0,250.0,300.0,803.554262,1246.842385,3101.719450,3.214217,2.487660,1.551659,...,0.015553,-499.215539,no spatter,117.952347,0.457778,817.866562,blank,Transition Mode,344.664948,NaN
299,500,70.0,250.0,300.0,764.237400,1162.501235,2949.956363,3.056950,2.537594,1.521126,...,0.020133,-456.050759,no spatter,101.102012,0.392381,710.030117,blank,Conduction Mode,340.301603,NaN


In [ ]:
# Assuming df is your DataFrame
columns_to_delete = ['ref','depth/thickness','length/width','width/depth', 'density kg/m^3','conductivity w/m.k','specific heat J/kg.k',
                     'surface temperature T_s , K', 'surface tension, N/m','Peak temp. , K','recoil pressure , pa', 'Unnamed: 20','spatter' ,'p','pi']  # List of columns to delete



# Deleting the specified columns
df1 = df.drop(columns=columns_to_delete, inplace=False)
df1

,Power,Velocity,layer thickness,beam D,depth of meltpool,width of melt pool,length of melt pool,meltpool shape,absorptivity,"surface tension force , N",surface tension / spot size,normalized enthalpy,maragoni number,aspect ratio length/depth,"solidification time , s",Weber number,VED,spatter.1,m,effective power
0,200,850.0,30.0,90.0,110.000000,140.000000,757.902572,LOF,0.511600,0.002374,27777.777780,8.762277,484.848485,6.890023,0.891650,0.762707,37.004658,blank,Conduction Mode,102.319994
1,200,1000.0,30.0,90.0,92.000000,130.000000,679.508923,LOF,0.470605,0.002134,27777.777780,7.431092,484.848485,7.385967,0.679509,0.946459,31.453959,blank,Conduction Mode,94.121044
2,200,1150.0,30.0,90.0,90.000000,105.000000,607.355145,LOF,0.434673,0.001894,27777.777780,6.400441,484.848485,6.748390,0.528135,1.118781,27.351269,blank,Conduction Mode,86.934645
3,200,1300.0,30.0,90.0,68.000000,105.000000,540.985910,Desired,0.403247,0.001701,27777.777780,5.584648,484.848485,7.955675,0.416143,1.273442,24.195353,blank,Conduction Mode,80.649475
4,200,1450.0,30.0,90.0,72.000000,105.000000,479.942895,LOF,0.375698,0.001524,27777.777780,4.926635,484.848485,6.665874,0.330995,1.405504,21.692386,blank,Conduction Mode,75.139529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,500,40.0,250.0,300.0,1005.779644,1474.095190,3570.517737,LOF,0.698683,0.012405,8333.333333,22.660410,2090.000000,3.550000,89.262943,0.007957,176.928521,blank,Transition Mode,349.341321
297,500,50.0,250.0,300.0,867.461958,1348.371226,3296.355442,Desired,0.695379,0.011431,8333.333333,20.172237,1840.000000,3.800000,65.927109,0.011478,141.542817,blank,Transition Mode,347.689251
298,500,60.0,250.0,300.0,803.554262,1246.842385,3101.719450,Desired,0.689330,0.010718,8333.333333,18.254473,1630.000000,3.860000,51.695324,0.015553,117.952347,blank,Transition Mode,344.664948
299,500,70.0,250.0,300.0,764.237400,1162.501235,2949.956363,Desired,0.680603,0.010153,8333.333333,16.686405,1430.000000,3.860000,42.142234,0.020133,101.102012,blank,Conduction Mode,340.301603


In [ ]:
# Assuming df is your DataFrame and 'melt pool shape' is the column you want to replace values in

# Define the mapping of values to be replaced
replacement_map = {'LOF': 0, 'keyhole': 1, 'Desired': 2, 'balling': 3}

# Replace values in the 'melt pool shape' column using the mapping
df1['meltpool shape'] = df1['meltpool shape'].replace(replacement_map)

# Display the DataFrame after replacement
print("DataFrame after replacing values in 'melt pool shape' column:")
df1

DataFrame after replacing values in 'melt pool shape' column:


,Power,Velocity,layer thickness,beam D,depth of meltpool,width of melt pool,length of melt pool,meltpool shape,absorptivity,"surface tension force , N",surface tension / spot size,normalized enthalpy,maragoni number,aspect ratio length/depth,"solidification time , s",Weber number,VED,spatter.1,m,effective power
0,200,850.0,30.0,90.0,110.000000,140.000000,757.902572,0,0.511600,0.002374,27777.777780,8.762277,484.848485,6.890023,0.891650,0.762707,37.004658,blank,Conduction Mode,102.319994
1,200,1000.0,30.0,90.0,92.000000,130.000000,679.508923,0,0.470605,0.002134,27777.777780,7.431092,484.848485,7.385967,0.679509,0.946459,31.453959,blank,Conduction Mode,94.121044
2,200,1150.0,30.0,90.0,90.000000,105.000000,607.355145,0,0.434673,0.001894,27777.777780,6.400441,484.848485,6.748390,0.528135,1.118781,27.351269,blank,Conduction Mode,86.934645
3,200,1300.0,30.0,90.0,68.000000,105.000000,540.985910,2,0.403247,0.001701,27777.777780,5.584648,484.848485,7.955675,0.416143,1.273442,24.195353,blank,Conduction Mode,80.649475
4,200,1450.0,30.0,90.0,72.000000,105.000000,479.942895,0,0.375698,0.001524,27777.777780,4.926635,484.848485,6.665874,0.330995,1.405504,21.692386,blank,Conduction Mode,75.139529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,500,40.0,250.0,300.0,1005.779644,1474.095190,3570.517737,0,0.698683,0.012405,8333.333333,22.660410,2090.000000,3.550000,89.262943,0.007957,176.928521,blank,Transition Mode,349.341321
297,500,50.0,250.0,300.0,867.461958,1348.371226,3296.355442,2,0.695379,0.011431,8333.333333,20.172237,1840.000000,3.800000,65.927109,0.011478,141.542817,blank,Transition Mode,347.689251
298,500,60.0,250.0,300.0,803.554262,1246.842385,3101.719450,2,0.689330,0.010718,8333.333333,18.254473,1630.000000,3.860000,51.695324,0.015553,117.952347,blank,Transition Mode,344.664948
299,500,70.0,250.0,300.0,764.237400,1162.501235,2949.956363,2,0.680603,0.010153,8333.333333,16.686405,1430.000000,3.860000,42.142234,0.020133,101.102012,blank,Conduction Mode,340.301603


In [ ]:
category_counts = df1['meltpool shape'].value_counts()

In [ ]:
category_counts

meltpool shape
0    159
1     56
2     55
3     31
Name: count, dtype: int64

In [ ]:
# Assuming df is your DataFrame and 'column1' and 'column2' are the columns you want to multiply

# Create a new column by multiplying 'column1' and 'column2'
df1['effective_power'] = df1['Power'] * df1['absorptivity']

# Display the DataFrame with the new column
print("DataFrame with the new column:")
df1


DataFrame with the new column:


,Power,Velocity,layer thickness,beam D,depth of meltpool,width of melt pool,length of melt pool,meltpool shape,absorptivity,"surface tension force , N",...,normalized enthalpy,maragoni number,aspect ratio length/depth,"solidification time , s",Weber number,VED,spatter.1,m,effective power,effective_power
0,200,850.0,30.0,90.0,110.000000,140.000000,757.902572,0,0.511600,0.002374,...,8.762277,484.848485,6.890023,0.891650,0.762707,37.004658,blank,Conduction Mode,102.319994,102.319994
1,200,1000.0,30.0,90.0,92.000000,130.000000,679.508923,0,0.470605,0.002134,...,7.431092,484.848485,7.385967,0.679509,0.946459,31.453959,blank,Conduction Mode,94.121044,94.121044
2,200,1150.0,30.0,90.0,90.000000,105.000000,607.355145,0,0.434673,0.001894,...,6.400441,484.848485,6.748390,0.528135,1.118781,27.351269,blank,Conduction Mode,86.934645,86.934645
3,200,1300.0,30.0,90.0,68.000000,105.000000,540.985910,2,0.403247,0.001701,...,5.584648,484.848485,7.955675,0.416143,1.273442,24.195353,blank,Conduction Mode,80.649475,80.649475
4,200,1450.0,30.0,90.0,72.000000,105.000000,479.942895,0,0.375698,0.001524,...,4.926635,484.848485,6.665874,0.330995,1.405504,21.692386,blank,Conduction Mode,75.139529,75.139529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,500,40.0,250.0,300.0,1005.779644,1474.095190,3570.517737,0,0.698683,0.012405,...,22.660410,2090.000000,3.550000,89.262943,0.007957,176.928521,blank,Transition Mode,349.341321,349.341321
297,500,50.0,250.0,300.0,867.461958,1348.371226,3296.355442,2,0.695379,0.011431,...,20.172237,1840.000000,3.800000,65.927109,0.011478,141.542817,blank,Transition Mode,347.689251,347.689251
298,500,60.0,250.0,300.0,803.554262,1246.842385,3101.719450,2,0.689330,0.010718,...,18.254473,1630.000000,3.860000,51.695324,0.015553,117.952347,blank,Transition Mode,344.664948,344.664948
299,500,70.0,250.0,300.0,764.237400,1162.501235,2949.956363,2,0.680603,0.010153,...,16.686405,1430.000000,3.860000,42.142234,0.020133,101.102012,blank,Conduction Mode,340.301603,340.301603


In [ ]:
# Define the mapping of values to be replaced
replacement_map = {'LOF': 0, 'keyhole': 1, 'Desired': 2, 'balling': 3}

# Replace values in the 'melt pool shape' column using the mapping
df1['meltpool shape'] = df1['meltpool shape'].replace(replacement_map)

# Display the DataFrame after replacement
print("DataFrame after replacing values in 'melt pool shape' column:")
df1

DataFrame after replacing values in 'melt pool shape' column:


,Power,Velocity,layer thickness,beam D,depth of meltpool,width of melt pool,length of melt pool,meltpool shape,absorptivity,"surface tension force , N",...,normalized enthalpy,maragoni number,aspect ratio length/depth,"solidification time , s",Weber number,VED,spatter.1,m,effective power,effective_power
0,200,850.0,30.0,90.0,110.000000,140.000000,757.902572,0,0.511600,0.002374,...,8.762277,484.848485,6.890023,0.891650,0.762707,37.004658,blank,Conduction Mode,102.319994,102.319994
1,200,1000.0,30.0,90.0,92.000000,130.000000,679.508923,0,0.470605,0.002134,...,7.431092,484.848485,7.385967,0.679509,0.946459,31.453959,blank,Conduction Mode,94.121044,94.121044
2,200,1150.0,30.0,90.0,90.000000,105.000000,607.355145,0,0.434673,0.001894,...,6.400441,484.848485,6.748390,0.528135,1.118781,27.351269,blank,Conduction Mode,86.934645,86.934645
3,200,1300.0,30.0,90.0,68.000000,105.000000,540.985910,2,0.403247,0.001701,...,5.584648,484.848485,7.955675,0.416143,1.273442,24.195353,blank,Conduction Mode,80.649475,80.649475
4,200,1450.0,30.0,90.0,72.000000,105.000000,479.942895,0,0.375698,0.001524,...,4.926635,484.848485,6.665874,0.330995,1.405504,21.692386,blank,Conduction Mode,75.139529,75.139529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,500,40.0,250.0,300.0,1005.779644,1474.095190,3570.517737,0,0.698683,0.012405,...,22.660410,2090.000000,3.550000,89.262943,0.007957,176.928521,blank,Transition Mode,349.341321,349.341321
297,500,50.0,250.0,300.0,867.461958,1348.371226,3296.355442,2,0.695379,0.011431,...,20.172237,1840.000000,3.800000,65.927109,0.011478,141.542817,blank,Transition Mode,347.689251,347.689251
298,500,60.0,250.0,300.0,803.554262,1246.842385,3101.719450,2,0.689330,0.010718,...,18.254473,1630.000000,3.860000,51.695324,0.015553,117.952347,blank,Transition Mode,344.664948,344.664948
299,500,70.0,250.0,300.0,764.237400,1162.501235,2949.956363,2,0.680603,0.010153,...,16.686405,1430.000000,3.860000,42.142234,0.020133,101.102012,blank,Conduction Mode,340.301603,340.301603


In [ ]:
columns_to_delete = ['Power','surface tension / spot size']  # List of columns to delete



# Deleting the specified columns
df2 = df1.drop(columns=columns_to_delete, inplace=False)
df2

,Velocity,layer thickness,beam D,depth of meltpool,width of melt pool,length of melt pool,meltpool shape,absorptivity,"surface tension force , N",normalized enthalpy,maragoni number,aspect ratio length/depth,"solidification time , s",Weber number,VED,spatter.1,m,effective power,effective_power
0,850.0,30.0,90.0,110.000000,140.000000,757.902572,0,0.511600,0.002374,8.762277,484.848485,6.890023,0.891650,0.762707,37.004658,blank,Conduction Mode,102.319994,102.319994
1,1000.0,30.0,90.0,92.000000,130.000000,679.508923,0,0.470605,0.002134,7.431092,484.848485,7.385967,0.679509,0.946459,31.453959,blank,Conduction Mode,94.121044,94.121044
2,1150.0,30.0,90.0,90.000000,105.000000,607.355145,0,0.434673,0.001894,6.400441,484.848485,6.748390,0.528135,1.118781,27.351269,blank,Conduction Mode,86.934645,86.934645
3,1300.0,30.0,90.0,68.000000,105.000000,540.985910,2,0.403247,0.001701,5.584648,484.848485,7.955675,0.416143,1.273442,24.195353,blank,Conduction Mode,80.649475,80.649475
4,1450.0,30.0,90.0,72.000000,105.000000,479.942895,0,0.375698,0.001524,4.926635,484.848485,6.665874,0.330995,1.405504,21.692386,blank,Conduction Mode,75.139529,75.139529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,40.0,250.0,300.0,1005.779644,1474.095190,3570.517737,0,0.698683,0.012405,22.660410,2090.000000,3.550000,89.262943,0.007957,176.928521,blank,Transition Mode,349.341321,349.341321
297,50.0,250.0,300.0,867.461958,1348.371226,3296.355442,2,0.695379,0.011431,20.172237,1840.000000,3.800000,65.927109,0.011478,141.542817,blank,Transition Mode,347.689251,347.689251
298,60.0,250.0,300.0,803.554262,1246.842385,3101.719450,2,0.689330,0.010718,18.254473,1630.000000,3.860000,51.695324,0.015553,117.952347,blank,Transition Mode,344.664948,344.664948
299,70.0,250.0,300.0,764.237400,1162.501235,2949.956363,2,0.680603,0.010153,16.686405,1430.000000,3.860000,42.142234,0.020133,101.102012,blank,Conduction Mode,340.301603,340.301603


In [ ]:
# # Assuming df is your DataFrame

# # Filter out rows where 'velocity' is greater than 2000
# filtered_df2 = df2[df2['Velocity'] <= 2000]

# # Display the filtered DataFrame
# print("Filtered DataFrame:")
# filtered_df2

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

In [ ]:
import pandas as pd

def detect_outliers_zscore_all(df, num_stds=3):
  """
  This function detects outliers based on z-scores within a specified number of standard deviations for all numerical columns in a DataFrame.

  Args:
      df (pandas.DataFrame): The DataFrame containing the data.
      num_stds (int, optional): The number of standard deviations to consider outliers. Defaults to 3.

  Returns:
      pandas.DataFrame: A DataFrame with columns indicating outliers (True) or inliers (False) for each numerical column.
      pandas.Index: The indices of rows containing outliers in any column.
  """
  outliers_df = pd.DataFrame(index=df.index)  # Create an empty DataFrame for outliers
  outlier_indices = []  # Initialize a list to store outlier row indices
  for col in df.select_dtypes(include=[np.number]):  # Iterate through numerical columns
    outliers = abs(df[col] - df[col].mean()) / df[col].std() > num_stds
    outliers_df[f'outliers_{col}'] = outliers  # Add a column named 'outliers_{col}' with outlier flags
    outlier_indices.extend(outliers[outliers].index)  # Add outlier indices to the list
  return outliers_df, pd.Index(outlier_indices)  # Return outlier flags DataFrame and outlier indices

# Assuming 'filtered_df2' is your prepared DataFrame and 'threshold' is defined

# Calculate z-scores and identify outlier indices
outliers_df, outlier_indices_zscore = detect_outliers_zscore_all(df2.copy())

# **Important: Consider if removing outliers is the best approach**
# (See previous discussion about potential drawbacks of removing outliers)

# If you decide to remove outliers:
# Drop the rows corresponding to the outlier indices
filtered_df_cleaned = df2.drop(outlier_indices_zscore)

# Display the cleaned DataFrame (optional)
print("DataFrame after removing outliers detected by z-score method:")
print(filtered_df_cleaned.shape)


DataFrame after removing outliers detected by z-score method:
(244, 19)


In [ ]:
# # Find the indices of rows where any z-score exceeds the threshold
# outliers_indices_zscore = z_scores.index[(z_scores > threshold).any(axis=1)]

# # Drop the rows corresponding to the outlier indices
# filtered_df_cleaned = filtered_df.drop(outliers_indices_zscore)

# # Display the cleaned DataFrame
# print("DataFrame after removing outliers detected by z-score method:")
# print(filtered_df_cleaned.shape)

In [ ]:
# Get the indices of outliers detected by Isolation Forest method
# outliers_indices_isolation_forest = filtered_df2.index[outliers_isolation_forest]

# # Drop the rows corresponding to the outlier indices
# filtered_df2_cleaned = filtered_df2.drop(outliers_indices_isolation_forest)

# # Display the cleaned DataFrame
# print("DataFrame after removing outliers detected by Isolation Forest method:")
# print(filtered_df2_cleaned)


In [ ]:
filtered_df_cleaned

,Velocity,layer thickness,beam D,depth of meltpool,width of melt pool,length of melt pool,meltpool shape,absorptivity,"surface tension force , N",normalized enthalpy,maragoni number,aspect ratio length/depth,"solidification time , s",Weber number,VED,spatter.1,m,effective power,effective_power
0,850.0,30.0,90.0,110.000000,140.000000,757.902572,0,0.511600,0.002374,8.762277,484.848485,6.890023,0.891650,0.762707,37.004658,blank,Conduction Mode,102.319994,102.319994
1,1000.0,30.0,90.0,92.000000,130.000000,679.508923,0,0.470605,0.002134,7.431092,484.848485,7.385967,0.679509,0.946459,31.453959,blank,Conduction Mode,94.121044,94.121044
2,1150.0,30.0,90.0,90.000000,105.000000,607.355145,0,0.434673,0.001894,6.400441,484.848485,6.748390,0.528135,1.118781,27.351269,blank,Conduction Mode,86.934645,86.934645
3,1300.0,30.0,90.0,68.000000,105.000000,540.985910,2,0.403247,0.001701,5.584648,484.848485,7.955675,0.416143,1.273442,24.195353,blank,Conduction Mode,80.649475,80.649475
4,1450.0,30.0,90.0,72.000000,105.000000,479.942895,0,0.375698,0.001524,4.926635,484.848485,6.665874,0.330995,1.405504,21.692386,blank,Conduction Mode,75.139529,75.139529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,140.0,100.0,80.0,409.132018,481.569429,1624.254112,0,0.699992,0.005334,39.655745,585.000000,3.970000,11.601815,0.044342,319.893085,blank,Transition Mode,157.498139,157.498139
277,170.0,100.0,80.0,414.076711,463.116369,1515.520762,0,0.699939,0.004998,35.984279,537.000000,3.660000,8.914828,0.061005,263.441364,blank,Transition Mode,157.486220,157.486220
278,200.0,100.0,80.0,310.431040,445.868551,1440.400024,0,0.699751,0.004759,33.166978,424.000000,4.640000,7.202000,0.080251,223.925159,blank,Transition Mode,157.444030,157.444030
279,290.0,100.0,80.0,273.390583,404.344250,1312.274798,0,0.697074,0.004333,27.438300,320.000000,4.800000,4.525086,0.153719,154.431144,blank,Transition Mode,156.841684,156.841684


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Assuming 'filtered_df_cleaned' is your cleaned DataFrame containing the data
# Assuming 'X' contains features and 'y_width' contains the target variable 'width of melt pool'

# Split the dataset into features (X) and the target variable (y_width)
X = filtered_df_cleaned.drop(columns=['depth of meltpool', 'width of melt pool', 'length of melt pool','meltpool shape','m','spatter.1','effective power'])  # Features excluding the target variable
y = filtered_df_cleaned['meltpool shape']  # Target variable

In [ ]:
X

,Velocity,layer thickness,beam D,absorptivity,"surface tension force , N",normalized enthalpy,maragoni number,aspect ratio length/depth,"solidification time , s",Weber number,VED,effective_power
0,850.0,30.0,90.0,0.511600,0.002374,8.762277,484.848485,6.890023,0.891650,0.762707,37.004658,102.319994
1,1000.0,30.0,90.0,0.470605,0.002134,7.431092,484.848485,7.385967,0.679509,0.946459,31.453959,94.121044
2,1150.0,30.0,90.0,0.434673,0.001894,6.400441,484.848485,6.748390,0.528135,1.118781,27.351269,86.934645
3,1300.0,30.0,90.0,0.403247,0.001701,5.584648,484.848485,7.955675,0.416143,1.273442,24.195353,80.649475
4,1450.0,30.0,90.0,0.375698,0.001524,4.926635,484.848485,6.665874,0.330995,1.405504,21.692386,75.139529
...,...,...,...,...,...,...,...,...,...,...,...,...
276,140.0,100.0,80.0,0.699992,0.005334,39.655745,585.000000,3.970000,11.601815,0.044342,319.893085,157.498139
277,170.0,100.0,80.0,0.699939,0.004998,35.984279,537.000000,3.660000,8.914828,0.061005,263.441364,157.486220
278,200.0,100.0,80.0,0.699751,0.004759,33.166978,424.000000,4.640000,7.202000,0.080251,223.925159,157.444030
279,290.0,100.0,80.0,0.697074,0.004333,27.438300,320.000000,4.800000,4.525086,0.153719,154.431144,156.841684


In [ ]:
y

0      0
1      0
2      0
3      2
4      0
      ..
276    0
277    0
278    0
279    0
280    2
Name: meltpool shape, Length: 244, dtype: int64

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
import pandas as pd

# Assuming y_test is a pandas Series or DataFrame containing the categories for your test data
test_category_counts = y_test.value_counts()

print("Number of data in each category in the test dataset:")
print(test_category_counts)


Number of data in each category in the test dataset:
meltpool shape
0    40
1    15
2     4
3     2
Name: count, dtype: int64


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the Random Forest classifier
rf_classifier = RandomForestClassifier()

# Define the hyperparameters grid for tuning
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Perform GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_

# Train the Random Forest classifier with the best hyperparameters
best_rf_classifier = RandomForestClassifier(**best_params)
best_rf_classifier.fit(X_train, y_train)

# Predictions on the test set
y_pred = best_rf_classifier.predict(X_test)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

# Classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Best Parameters:", best_params)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Fitting 5 folds for each of 81 candidates, totalling 405 fits
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.97      0.97        33
           1       0.91      0.91      0.91        11
           2       0.67      0.67      0.67         3
           3       1.00      1.00      1.00         2

    accuracy                           0.94        49
   macro avg       0.89      0.89      0.89        49
weighted avg       0.94      0.94      0.94        49

Best Parameters: {'max_depth': 20, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 100}
Accuracy: 0.9387755102040817
Precision: 0.8863636363636364
Recall: 0.8863636363636364
F1 Score: 0.8863636363636364


# **ADABOOST**



In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the AdaBoost classifier
adaboost_classifier = AdaBoostClassifier()

# Define the hyperparameters grid for tuning
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.1, 0.5, 1.0]
}

# Perform GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(estimator=adaboost_classifier, param_grid=param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_

# Train the AdaBoost classifier with the best hyperparameters
best_adaboost_classifier = AdaBoostClassifier(**best_params)
best_adaboost_classifier.fit(X_train, y_train)

# Predictions on the test set
y_pred = best_adaboost_classifier.predict(X_test)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

# Classification report
print("AdaBoost Classifier:")
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Best Parameters:", best_params)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Fitting 5 folds for each of 9 candidates, totalling 45 fits
AdaBoost Classifier:
Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.97      0.96        33
           1       1.00      0.64      0.78        11
           2       0.29      0.67      0.40         3
           3       1.00      0.50      0.67         2

    accuracy                           0.86        49
   macro avg       0.81      0.69      0.70        49
weighted avg       0.92      0.86      0.87        49

Best Parameters: {'learning_rate': 0.1, 'n_estimators': 100}
Accuracy: 0.8571428571428571
Precision: 0.8067226890756303
Recall: 0.6931818181818181
F1 Score: 0.6999170812603648


In [ ]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 6.3 MB/s eta 0:00:00


In [ ]:
!pip install xgboost

In [ ]:
pip show catboost


Name: catboost
Version: 1.2.5
Summary: CatBoost Python Package
Home-page: https://catboost.ai
Author: CatBoost Developers
Author-email: 
License: Apache License, Version 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: graphviz, matplotlib, numpy, pandas, plotly, scipy, six
Required-by: 


In [ ]:
pip install --upgrade catboost


# catboost

In [ ]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# Assuming 'X' contains features and 'y' contains the target variable

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


# CatBoost Classifier
catboost_classifier = CatBoostClassifier()

catboost_param_grid = {
    'iterations': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'depth': [4, 6, 8]
}

catboost_grid_search = GridSearchCV(estimator=catboost_classifier, param_grid=catboost_param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)
catboost_grid_search.fit(X_train, y_train)
best_catboost_params = catboost_grid_search.best_params_
best_catboost_model = catboost_grid_search.best_estimator_


# Evaluation Metrics
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')
    return accuracy, precision, recall, f1



# Evaluate XGBoost Classifier
xgboost_accuracy, xgboost_precision, xgboost_recall, xgboost_f1 = evaluate_model(best_xgboost_model, X_test, y_test)




print("XGBoost Classifier:")
print("Best Parameters:", best_xgboost_params)
print("Accuracy:", xgboost_accuracy)
print("Precision:", xgboost_precision)
print("Recall:", xgboost_recall)
print("F1 Score:", xgboost_f1)
print()


Fitting 5 folds for each of 9 candidates, totalling 45 fits
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 27 candidates, totalling 135 fits
0:	learn: 1.2961720	total: 50ms	remaining: 2.45s
1:	learn: 1.1938256	total: 53ms	remaining: 1.27s
2:	learn: 1.1169082	total: 57.2ms	remaining: 897ms
3:	learn: 1.0474077	total: 60.8ms	remaining: 699ms
4:	learn: 0.9793916	total: 65.1ms	remaining: 586ms
5:	learn: 0.9241115	total: 68.5ms	remaining: 503ms
6:	learn: 0.8736969	total: 72.8ms	remaining: 447ms
7:	learn: 0.8317415	total: 76.4ms	remaining: 401ms
8:	learn: 0.7877261	total: 80.6ms	remaining: 367ms
9:	learn: 0.7670927	total: 84ms	remaining: 336ms
10:	learn: 0.7359633	total: 88.3ms	remaining: 313ms
11:	learn: 0.7155719	total: 92.5ms	remaining: 293ms
12:	learn: 0.6888627	total: 97.3ms	remaining: 277ms
13:	learn: 0.6727047	total: 101ms	remaining: 261ms
14:	learn: 0.6474610	total: 106ms	remaining: 246ms
15:	learn: 0.6269644	total: 108ms	remaining: 230ms
16:

In [ ]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# Assuming 'X' contains features and 'y' contains the target variable

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


# CatBoost Classifier
catboost_classifier = CatBoostClassifier()

catboost_param_grid = {
    'iterations': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'depth': [4, 6, 8]
}

catboost_grid_search = GridSearchCV(estimator=catboost_classifier, param_grid=catboost_param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)
catboost_grid_search.fit(X_train, y_train)
best_catboost_params = catboost_grid_search.best_params_
best_catboost_model = catboost_grid_search.best_estimator_


# Evaluation Metrics
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')
    return accuracy, precision, recall, f1



# Evaluate XGBoost Classifier
xgboost_accuracy, xgboost_precision, xgboost_recall, xgboost_f1 = evaluate_model(best_xgboost_model, X_test, y_test)




print("XGBoost Classifier:")
print("Best Parameters:", best_xgboost_params)
print("Accuracy:", xgboost_accuracy)
print("Precision:", xgboost_precision)
print("Recall:", xgboost_recall)
print("F1 Score:", xgboost_f1)
print()


Fitting 5 folds for each of 27 candidates, totalling 135 fits


In [ ]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Lasso Classifier
lasso_classifier = LogisticRegression(penalty='l1', solver='liblinear')

lasso_param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100]
}

lasso_grid_search = GridSearchCV(estimator=lasso_classifier, param_grid=lasso_param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)
lasso_grid_search.fit(X_train, y_train)
best_lasso_params = lasso_grid_search.best_params_
best_lasso_model = lasso_grid_search.best_estimator_

# Ridge Classifier
ridge_classifier = LogisticRegression(penalty='l2', solver='liblinear')

ridge_param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100]
}

ridge_grid_search = GridSearchCV(estimator=ridge_classifier, param_grid=ridge_param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)
ridge_grid_search.fit(X_train, y_train)
best_ridge_params = ridge_grid_search.best_params_
best_ridge_model = ridge_grid_search.best_estimator_

# Elastic Net Classifier
elasticnet_classifier = LogisticRegression(penalty='elasticnet', solver='saga', l1_ratio=0.5)

elasticnet_param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9]
}

elasticnet_grid_search = GridSearchCV(estimator=elasticnet_classifier, param_grid=elasticnet_param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)
elasticnet_grid_search.fit(X_train, y_train)
best_elasticnet_params = elasticnet_grid_search.best_params_
best_elasticnet_model = elasticnet_grid_search.best_estimator_

# Evaluation Metrics
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')
    return accuracy, precision, recall, f1

# Evaluate Lasso Classifier
lasso_accuracy, lasso_precision, lasso_recall, lasso_f1 = evaluate_model(best_lasso_model, X_test, y_test)

# Evaluate Ridge Classifier
ridge_accuracy, ridge_precision, ridge_recall, ridge_f1 = evaluate_model(best_ridge_model, X_test, y_test)

# Evaluate Elastic Net Classifier
elasticnet_accuracy, elasticnet_precision, elasticnet_recall, elasticnet_f1 = evaluate_model(best_elasticnet_model, X_test, y_test)

# Print results
print("Lasso Classifier:")
print("Best Parameters:", best_lasso_params)
print("Accuracy:", lasso_accuracy)
print("Precision:", lasso_precision)
print("Recall:", lasso_recall)
print("F1 Score:", lasso_f1)
print()

print("Ridge Classifier:")
print("Best Parameters:", best_ridge_params)
print("Accuracy:", ridge_accuracy)
print("Precision:", ridge_precision)
print("Recall:", ridge_recall)
print("F1 Score:", ridge_f1)
print()

print("Elastic Net Classifier:")
print("Best Parameters:", best_elasticnet_params)
print("Accuracy:", elasticnet_accuracy)
print("Precision:", elasticnet_precision)
print("Recall:", elasticnet_recall)
print("F1 Score:", elasticnet_f1)


Fitting 5 folds for each of 6 candidates, totalling 30 fits
Fitting 5 folds for each of 6 candidates, totalling 30 fits
Fitting 5 folds for each of 30 candidates, totalling 150 fits
Lasso Classifier:
Best Parameters: {'C': 100}
Accuracy: 0.9016393442622951
Precision: 0.7199477351916376
Recall: 0.7729166666666667
F1 Score: 0.7363072432037949

Ridge Classifier:
Best Parameters: {'C': 100}
Accuracy: 0.9016393442622951
Precision: 0.7199477351916376
Recall: 0.7729166666666667
F1 Score: 0.7363072432037949

Elastic Net Classifier:
Best Parameters: {'C': 0.1, 'l1_ratio': 0.1}
Accuracy: 0.819672131147541
Precision: 0.731416957026713
Recall: 0.7666666666666666
F1 Score: 0.7374931581828135


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [ ]:
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF, Matern, RationalQuadratic, Product, Sum
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the kernels and their respective parameter grids
kernels = [
    (RBF(length_scale=1.0), {'kernel__length_scale': [0.1, 1.0, 10.0]}),
    (Matern(length_scale=1.0), {'kernel__length_scale': [0.1, 1.0, 10.0], 'kernel__nu': [0.5, 1.5, 2.5]}),
    (RationalQuadratic(length_scale=1.0, alpha=0.1), {'kernel__length_scale': [0.1, 1.0, 10.0], 'kernel__alpha': [0.1, 0.5, 1.0]}),
    (Product(RBF(length_scale=1.0), RBF(length_scale=1.0)), {'kernel__k1__length_scale': [0.1, 1.0, 10.0], 'kernel__k2__length_scale': [0.1, 1.0, 10.0]}),
    (Sum(RBF(length_scale=1.0), RBF(length_scale=1.0)), {'kernel__k1__length_scale': [0.1, 1.0, 10.0], 'kernel__k2__length_scale': [0.1, 1.0, 10.0]}),
]

# Train and evaluate the classifier for each kernel
for kernel, param_grid in kernels:
    gpc_classifier = GaussianProcessClassifier(kernel=kernel, random_state=42)

    # Perform GridSearchCV to find the best parameters
    grid_search = GridSearchCV(gpc_classifier, param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)
    best_params = grid_search.best_params_

    # Train the classifier with the best parameters
    best_gpc_classifier = GaussianProcessClassifier(kernel=kernel, random_state=42)
    best_gpc_classifier.set_params(**best_params)
    best_gpc_classifier.fit(X_train, y_train)
    y_pred = best_gpc_classifier.predict(X_test)

    # Calculate evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')

    # Print evaluation metrics and best parameters
    print("Kernel:", kernel)
    print("Best Parameters:", best_params)
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)
    print()


Kernel: RBF(length_scale=10)
Best Parameters: {'kernel__length_scale': 10.0}
Accuracy: 0.8163265306122449
Precision: 0.8983918128654971
Recall: 0.6022727272727273
F1 Score: 0.6849765258215962

Kernel: Matern(length_scale=10, nu=1.5)
Best Parameters: {'kernel__length_scale': 10.0, 'kernel__nu': 1.5}
Accuracy: 0.8163265306122449
Precision: 0.8983918128654971
Recall: 0.6022727272727273
F1 Score: 0.6849765258215962



/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter alpha is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter alpha is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter alpha is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value fo

Kernel: RationalQuadratic(alpha=0.1, length_scale=0.1)
Best Parameters: {'kernel__alpha': 0.1, 'kernel__length_scale': 0.1}
Accuracy: 0.8775510204081632
Precision: 0.9615384615384616
Recall: 0.6401515151515151
F1 Score: 0.731359649122807

Kernel: RBF(length_scale=10) * RBF(length_scale=10)
Best Parameters: {'kernel__k1__length_scale': 10.0, 'kernel__k2__length_scale': 10.0}
Accuracy: 0.8163265306122449
Precision: 0.8983918128654971
Recall: 0.6022727272727273
F1 Score: 0.6849765258215962

Kernel: RBF(length_scale=10) + RBF(length_scale=10)
Best Parameters: {'kernel__k1__length_scale': 10.0, 'kernel__k2__length_scale': 10.0}
Accuracy: 0.8367346938775511
Precision: 0.9094594594594595
Recall: 0.625
F1 Score: 0.7035714285714286



In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Define the KNN classifier
knn_classifier = KNeighborsClassifier()

# Define the hyperparameters grid for tuning
param_grid = {
    'n_neighbors': [3, 5, 7],           # Number of neighbors
    'weights': ['uniform', 'distance'], # Weight function used in prediction
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'], # Algorithm used to compute the nearest neighbors
    'p': [1, 2]                         # Power parameter for the Minkowski metric
}

# Perform GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(estimator=knn_classifier, param_grid=param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_

# Train the KNN classifier with the best hyperparameters
best_knn_classifier = KNeighborsClassifier(**best_params)
best_knn_classifier.fit(X_train, y_train)

# Predictions on the test set
y_pred = best_knn_classifier.predict(X_test)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

# Print evaluation metrics and best parameters
print("Best Parameters:", best_params)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Fitting 5 folds for each of 48 candidates, totalling 240 fits
Best Parameters: {'algorithm': 'auto', 'n_neighbors': 7, 'p': 1, 'weights': 'distance'}
Accuracy: 0.8524590163934426
Precision: 0.7917862266857962
Recall: 0.6354166666666667
F1 Score: 0.6815200617283951


In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the Neural Network classifier
nn_classifier = MLPClassifier()

# Define the hyperparameters grid for tuning
param_grid = {
    'hidden_layer_sizes': [(100,), (50, 50), (100, 50, 25)],  # Size of hidden layers
    'activation': ['logistic', 'tanh', 'relu'],               # Activation function for the hidden layer
    'solver': ['sgd', 'adam'],                                # Solver for weight optimization
    'alpha': [0.0001, 0.001, 0.01],                           # L2 penalty (regularization term)
    'learning_rate': ['constant', 'invscaling', 'adaptive'],  # Learning rate schedule
    'max_iter': [200, 300, 400]                               # Maximum number of iterations
}

# Perform GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(estimator=nn_classifier, param_grid=param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_

# Train the Neural Network classifier with the best hyperparameters
best_nn_classifier = MLPClassifier(**best_params)
best_nn_classifier.fit(X_train, y_train)

# Predictions on the test set
y_pred = best_nn_classifier.predict(X_test)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

# Print evaluation metrics and best parameters
print("Best Parameters:", best_params)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Fitting 5 folds for each of 486 candidates, totalling 2430 fits


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  warnings.warn(


Best Parameters: {'activation': 'logistic', 'alpha': 0.001, 'hidden_layer_sizes': (100, 50, 25), 'learning_rate': 'constant', 'max_iter': 400, 'solver': 'adam'}
Accuracy: 0.9387755102040817
Precision: 0.8772727272727273
Recall: 0.9621212121212122
F1 Score: 0.9069602272727273


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  warnings.warn(
